# rush-py

> Python SDK for the QDX Rush quantum computational chemistry workflow management system

# Quickstart
This document will walk through executing jobs on the Rush platform. For a comprehensive guide on the concepts and constructing a full workflow, see the [full rush-py explainer](https://talo.github.io/rush-py/full-rush-py-explainer.html) document.

First, install the following modules via pip—we require Python ≥ 3.9:
```
pip install rush-py pdb-tools
```

# 0) Code Sample
See the detailed breakdown in sections.

*_NOTE_*: This assumes that you are running code in a Jupyter notebook, which allows for top level `await` calls. If you are writing a normal Python script, you will need to wrap your code in something like the following:
``` python
import asyncio
async def main():
    #your code here
asyncio.run(main)
```

In [ ]:
# |hide
!rm -r ./.rush

In [ ]:
# Get a pdb to work with - we use the pdb-tools cli here but you can download directly from rcsb.org
!pdb_fetch 1brs | pdb_selchain -A | pdb_delhetatm > 1B39_A_nohet.pdb

In [ ]:
#...import the dependencies and set your configuration
import os
from pathlib import Path
import rush
import asyncio

RUSH_TOKEN = os.getenv("RUSH_TOKEN") or "YOUR_TOKEN_HERE"

# 1.3 Build your client
client = await rush.build_provider_with_functions(access_token=RUSH_TOKEN)

# 2.1 Prepare the protein
prepared_protein_qdxf, prepared_protein_pdb = await client.prepare_protein(Path("1B39_A_nohet.pdb"))

# 2.3 Return run values
protein_qdxf_value = await prepared_protein_qdxf.get()

2024-02-25 16:00:51,761 - rush - INFO - Argument acf2ed7f-401c-4791-a271-7a0a064ef9a6 is now ModuleInstanceStatus.RESOLVING
2024-02-25 16:00:53,950 - rush - INFO - Argument acf2ed7f-401c-4791-a271-7a0a064ef9a6 is now ModuleInstanceStatus.ADMITTED
2024-02-25 16:00:59,402 - rush - INFO - Argument acf2ed7f-401c-4791-a271-7a0a064ef9a6 is now ModuleInstanceStatus.RUNNING
2024-02-25 16:01:06,397 - rush - INFO - Argument acf2ed7f-401c-4791-a271-7a0a064ef9a6 is now ModuleInstanceStatus.AWAITING_UPLOAD


In [ ]:
# |hide
!rm 1B39_A_nohet.pdb

# 1) Setup
This is where we prepare the rush client, directories, and input data we'll be working with.

## 1.0) Imports

In [ ]:
import os
import tarfile
from datetime import datetime
from pathlib import Path

import py3Dmol
import requests
from pdbtools import pdb_delhetatm, pdb_fetch, pdb_selchain

import rush

## 1.1) Credentials
Retrieve your API token from the [Rush UI](https://rush.qdx.co/dashboard/settings).

You can either set the `RUSH_URL` and `RUSH_TOKEN` environment variables or provide them as variables to the client directly.

To see how to set environment variables, [Wikipedia](https://en.wikipedia.org/wiki/Environment_variable) has an extensive article.

In [ ]:
RUSH_URL = os.getenv("RUSH_URL") or "https://tengu.qdx.ai"
RUSH_TOKEN = os.getenv("RUSH_TOKEN") or "YOUR_TOKEN_HERE"

## 1.2) Configuration
Lets set some global variables that define our project. These are not required, but are good practice to help organize the jobs that will be persisted under your account.

Make sure you create a unique set of tags for each run.
Good practice is to have at least each of the experiment name and system name as a tag.

In [ ]:
EXPERIMENT = "rush-py-quickstart"
SYSTEM = "1B39"
TAGS = ["qdx", EXPERIMENT, SYSTEM]

In [ ]:
# |hide
WORK_DIR = Path.home() / "qdx" / EXPERIMENT

if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)

os.makedirs(WORK_DIR, exist_ok=True)
import sys

os.chdir(WORK_DIR)

## 1.3) Build your client
Get our client, which we'll use for calling modules and generally for using the Rush API.

As mentioned earlier, `url` and `access_token` are optional if you have set the env variables `RUSH_URL` and `RUSH_TOKEN` respectively.

`batch_tags` will be applied to each run that is spawned by this client.

A folder called `.rush` will be created in your workspace directory (defaults to the current working directory, can be overridden by passing `workspace=` to the provider builder).

In [ ]:
# By using the `build_provider_with_functions` method,
# we will also build helper functions calling each module
client = await rush.build_provider_with_functions(url=RUSH_URL, access_token=RUSH_TOKEN, batch_tags=TAGS)

In [ ]:
# |hide
client = await rush.build_provider_with_functions(
    url=RUSH_URL, access_token=RUSH_TOKEN, batch_tags=TAGS, restore_by_default=True
)

## 1.4) Input selection
Fetch data files from RCSB to pass as input to the modules:

In [ ]:
PROTEIN_PDB_PATH = client.workspace / f"{SYSTEM}_P.pdb"

complex = list(pdb_fetch.fetch_structure(SYSTEM))
protein = pdb_delhetatm.remove_hetatm(pdb_selchain.select_chain(complex, "A"))
with open(PROTEIN_PDB_PATH, "w") as f:
    for l in protein:
        f.write(str(l))

In [ ]:
help(client.convert)

Help on function convert in module rush.provider:

async convert(*args: *tuple[EnumValue, RushObject[NoneType]], target: 'Target | None' = None, resources: 'Resources | None' = {'storage': 10, 'storage_units': 'MB', 'gpus': 0}, tags: 'list[str] | None' = None, restore: 'bool | None' = None) -> tuple[RushObject[list[Record]]]
    Convert biomolecular and chemical file formats to the QDX file format. Supports PDB and SDF
    
    Module version:  
    `github:talo/tengu-prelude/6ce15cd56dc7a3cec8e5e6951bb473e33c5ecd11#convert`
    
    QDX Type Description:
    
        format: Format[PDB  |  SDF];
        input: Object{path: @$Bytes,  size: u32}
        ->
        output: Object{
            path: @[Conformer{
                residue_seq: [string]?,
                 residue_seq_ids: [i32]?,
                 residue_insertion_codes: [string]?,
                 topology: Topology{
                      version: TopologyVersion[V0  |  V1]?,
                      geometry: [f32],
          

# 2) Running Rush Modules
You can view which modules are available, alongside their documentation, in the [API Documentation](https://talo.github.io/rush-py/api/).

## 2.0) Prep the protein
First we will run the protein preparation routine (using pdbfixer and pdb2pqr internally) to prepare the protein for a molecular dynamics simulation.

In [ ]:
# we can check the arguments and outputs for prepare_protein with help()
help(client.prepare_protein)

Help on function prepare_protein in module rush.provider:

async prepare_protein(*args: *tuple[RushObject[NoneType]], target: 'Target | None' = None, resources: 'Resources | None' = {'storage': 138, 'storage_units': 'MB', 'gpus': 1}, tags: 'list[str] | None' = None, restore: 'bool | None' = None) -> tuple[RushObject[list[Record]], RushObject[NoneType]]
    Prepare a PDB for downstream tasks: protonate, fill missing atoms, etc.
    
    Module version:  
    `github:talo/prepare_protein/58bca0b416a7dd2080cf40d54cd91d619284407b#prepare_protein_tengu`
    
    QDX Type Description:
    
        input_pdb: Object{size: u32,  path: @$Bytes}
        ->
        output_qdxf: Object{
            size: u32,
             path: @[Conformer{
                 residue_insertion_codes: [string]?,
                  residue_seq_ids: [i32]?,
                  amino_acid_seq_ids: [i32]?,
                  binding_site_interactions: [BindingSiteInteraction{
                      ligand_atom_idx: u32,
     

In [ ]:
# Here we run the function, it will return a Provider.Arg which you can use to
# fetch the results
# We set restore = True so that we can restore a previous run to the same path
# with the same tags
prepared_protein_qdxf, prepared_protein_pdb = await client.prepare_protein(
    PROTEIN_PDB_PATH,
)
# This initially only has the id of your result; we will show how to fetch the
# actual value later
prepared_protein_qdxf

2024-02-25 16:01:27,803 - rush - INFO - Trying to restore job with tags: ['qdx', 'rush-py-quickstart', '1B39'] and path: github:talo/prepare_protein/58bca0b416a7dd2080cf40d54cd91d619284407b#prepare_protein_tengu
2024-02-25 16:01:27,869 - rush - INFO - Restoring job from previous run with id 7d56b888-a40b-4e70-9e57-2bba365a99c9


Arg(id=f4ef6699-fa8c-4b77-8dcd-ec8e69cbb5c6, value=None)

## 2.1) Run statuses
This will show the status of all of your runs. You can also view run statuses on the [Rush UI](https://rush.qdx.co/dashboard/jobs).

In [ ]:
await client.status()

{}

## 2.2) Run Values
This will return the "value" of the output from the function—for files you will recieve a url that you can download, otherwise you will recieve them as python types:

In [ ]:
protein_qdxf_value = await prepared_protein_qdxf.get()
protein_qdxf_value

{'url': 'https://storage.googleapis.com/qdx-store/a8934d65-ee56-4678-8e06-8ee9ffb7f9bd?x-goog-signature=3edba448950aa1a11364802ce4d87ab369b6dac920f5bfe6820476ec2059af2bcf68205511971f68b422931f5d61b0477f70fb12350d0ae39e8686dc274d74b5c178856ce549a952f37554d20e20415a0c1ed30459d55b87ef31d2beea30a65e90b562ca6cb75a7d800abf1c47ea01ddfb945c1e976efd6acc4e2c015b95e9d4eb8358acfcc71b5d2b2178736427239fd7b6c4c9b35534ae1339726f43d37145fab89f374bc9629e07113fccd796eca53ad49e06eca4ef462a32078a1717ed3853a1742336c9e06e249d247d4536517931d7781fdfba6fbb613853e370c96f0b7d4b610ec7cff76f1f023e02dfd62585a808237c771bef6de5e32175a3d57a05&x-goog-algorithm=GOOG4-RSA-SHA256&x-goog-credential=qdx-store-user%40humming-bird-321603.iam.gserviceaccount.com%2F20240225%2Fasia-southeast1%2Fstorage%2Fgoog4_request&x-goog-date=20240225T080129Z&x-goog-expires=3600&x-goog-signedheaders=host'}

## 2.3) Downloads
We provide a utility to download files into your workspace, you can either provide a filename, which will be saved in `workspace/objects/[filename]`, or you can provide your own filepath which the client will use as-is:

In [ ]:
await prepared_protein_pdb.download(filename="01_prepared_protein.pdb", overwrite=True)

In [ ]:
# we can read our prepared protein pdb like this
with open(client.workspace / "objects" / "01_prepared_protein.pdb", "r") as f:
    print(f.readline(), "...")

"..." ...
